APUNTES:DFUNCION 
cv.findContours()Esta función es clave para detectar bordes y formas en imágenes binarias.
2º Parámetro: El Modo (Jerarquía)Este parámetro decide cómo se relacionan los contornos entre sí (si unos están dentro de otros).cv.RETR_LIST (El más común)Significado: "Dame una lista plana de todos los contornos que veas. No me importa quién está dentro de quién".Resultado: Te devuelve todos los bordes por igual (ej: el borde exterior de una rosquilla y el agujero interior son dos elementos independientes). No distingue "padres" ni "hijos".cv.RETR_TREE (El completo)Significado: "Reconstruye toda la jerarquía de familia".Resultado: Te dice exactamente quién contiene a quién (ej: "El borde A está dentro del B, y el B está dentro del C"). Útil si quieres ignorar agujeros internos.cv.RETR_EXTERNAL (El superficial)Significado: "Dame solo los contornos de fuera. Ignora lo que haya dentro".Resultado: Solo te daría el borde exterior de la rosquilla. El agujero lo ignora por completo.3º Parámetro: La Aproximación (Optimización)Este parámetro decide cuánta memoria gastamos guardando los puntos del contorno.cv.CHAIN_APPROX_NONE (El bruto)Significado: "Guarda TODOS los puntos del borde, píxel a píxel".Resultado: Si tienes una línea recta de 100 píxeles, te guardará 100 coordenadas (x1,y1), (x2,y1).... Gasta mucha memoria innecesaria.cv.CHAIN_APPROX_SIMPLE (El inteligente - Recomendado)Significado: "Guarda solo los puntos esenciales para reconstruir la forma".Resultado: Comprime el contorno.Línea recta: Solo guarda el punto inicial y el final (elimina los del medio).Cuadrado: Solo guarda las 4 esquinas (en lugar de los miles de puntos del perímetro).

-----------------
Fai un programa en Python que coa imaxe escaneada:

- Identifica os cuadrilateros da imaxe.

- Conta o número de "circulos" e encerraos dentro dun cuadrilatero

In [9]:
from matplotlib import pyplot as plt
import numpy as numpy
import cv2 as cv

# opencv fondo negro y tal siempre inventir la img , si esta invertia
img = cv.imread("../../docs/Escaneo-cargador.jpg",0)
#2ºparam El Modo , Este parámetro decide cómo se relacionan los contornos entre sí
cnts,hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE )
display(f"Numero de contornos detectados {len(cnts)}")

conContornos = cv.cvtColor(img,cv.COLORMAP_PINK)
cv.drawContours(conContornos, cnts, -1, (0, 255, 0), 2)
plt.subplot(121),plt.imshow(conContornos,cmap = 'gray')
plt.subplot(121),plt.imshow(img,cmap = 'gray')


# plt.imshow(img)


'Numero de contornos detectados 1'

error: OpenCV(4.13.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-15:Bad number of channels) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<1>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = cv::impl::<unnamed>::NONE; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
